<a href="https://colab.research.google.com/github/desaiankitb/pytorch-basics/blob/main/pytorch-with-examples/02_polynomial_autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

# PyTorch: Tensors and autograd

A third order polynomial, trained to predict $y = sin(x)$ from $-\pi$ to $\pi$ by minimizing squared Euclidean distance. 

This implementation computes the forward pass using operations on PyTorch Tensors, and uses PyTorch autograd to compute gradients. 

A PyTorch Tensor represents a node in a computational graph. If `x` is a Tensor that has `x.required_grad=True` then `x.grad` is another Tensor holding the gradient of `x` with respect to some scalar value. 

In [8]:
import torch
import math 

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create Tensors to hold input and outputs. 
# By default, required_grad=False, which indicates that we do not need to 
# compute gradients with respect to these Tensors during the backward pass. 
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensor for weights. For a third order polynomial, we need 
# 4 weights, y = a + b x + c x^2 + d x^3 
# Setting required_grad=True indicates that we want to compute gradients 
# with respect to these Tensors during backward pass. 

a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)  
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
  # Forward pass: compute predicted y using operations on Tensors. 
  y_pred = a + b * x + c * x ** 2 + d * x ** 3 

  # Compute and print loss using operations on Tensors. 
  # Now loss is a Tensor of shape (1,)
  # loss.item() gets the scalar value held in the loss. 
  loss = (y_pred-y).pow(2).sum()
  if t % 100 == 99:
    print(t, loss.item())

  # Use autograd to comptute the backward pass. This call will compute the 
  # gradient of loss with respect to all Tensors with required_grad=True. 
  # After this call a.grad, b.grad, c.grad, d.grad will be Tensors holding 
  # the gradient of the loss with respect to a, b, c, d, respectively. 
  loss.backward()

  # Manually update weights using gradient descent. Wrap in torch.no_grad()
  # because weights have requires_grad=True, but we don't need to track this 
  # in autograd. 
  with torch.no_grad():
    a -= learning_rate * a.grad
    b -= learning_rate * b.grad 
    c -= learning_rate * c.grad
    d -= learning_rate * d.grad 

    # Manually zero the gradients after updating weights 
    a.grad = None 
    b.grad = None 
    c.grad = None 
    d.grad = None 

print(f"Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3")



99 891.4742431640625
199 624.8438110351562
299 439.07208251953125
399 309.5333251953125
499 219.13711547851562
599 156.00892639160156
699 111.89220428466797
799 81.0406494140625
899 59.451629638671875
999 44.33494567871094
1099 33.743896484375
1199 26.319400787353516
1299 21.111927032470703
1399 17.45757293701172
1499 14.891921997070312
1599 13.089749336242676
1699 11.823325157165527
1799 10.933008193969727
1899 10.306864738464355
1999 9.866337776184082
Result: y = 0.03309646621346474 + 0.8485207557678223 x + -0.005709691904485226 x^2 + -0.0921611487865448 x^3
